In [1]:
#imports
import pandas as pd
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 40)
import hyperopt
from hyperopt import hp, tpe, STATUS_OK, Trials
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import pickle
import time
import numpy as np
import xgboost as xgb

In [2]:
X_MAIN=pd.read_csv('prepped_train.csv')
y_MAIN=X_MAIN['num_orders']

In [3]:
test=pd.read_csv('prepped_test.csv')

In [4]:
# find unique combos for seperate feature creation
g1=test.groupby(by=['center_id','meal_id']).count()
g1=g1.add_suffix('_Count').reset_index()
ls=[]
for i in range(g1.shape[0]):   
    ls.append((g1['center_id'][i],g1['meal_id'][i]))

In [6]:
# data prep by combining from all combos
data_fold1=pd.DataFrame()
data_fold2=pd.DataFrame()
data_fold3=pd.DataFrame()
data_fold4=pd.DataFrame()
data_fold5=pd.DataFrame()
valid_fold1=pd.DataFrame()
valid_fold2=pd.DataFrame()
valid_fold3=pd.DataFrame()
valid_fold4=pd.DataFrame()
valid_fold5=pd.DataFrame()
cc=0
for j in ls:
    cc=cc+1
    X=X_MAIN[(X_MAIN["center_id"]==j[0]) & (X_MAIN['meal_id']==j[1])]
    y=X['num_orders']
    skf = TimeSeriesSplit(n_splits=5)
    count=0
    try:
        for train_index, test_index in skf.split(X[0:X.shape[0]], y):
            count=count+1
            X_train, X_test = pd.DataFrame(X.values[train_index],columns=X_MAIN.columns,dtype=int), pd.DataFrame(X.values[test_index],columns=X_MAIN.columns,dtype=int)
            minn=X_test['week'].min()
            X_train=X_train[X_train['week']<=minn-10]
            X_test=X_test
            if count==1:
                data_fold1=data_fold1.append(X_train)
                valid_fold1=valid_fold1.append(X_test)
            elif count==2:
                data_fold2=data_fold2.append(X_train)
                valid_fold2=valid_fold2.append(X_test)
            elif count==3:
                data_fold3=data_fold3.append(X_train)
                valid_fold3=valid_fold3.append(X_test)
            elif count==4:
                data_fold4=data_fold4.append(X_train)
                valid_fold4=valid_fold4.append(X_test)
            else:
                data_fold5=data_fold5.append(X_train)
                valid_fold5=valid_fold5.append(X_test)
    except:
        print(j)
        continue

(41, 1770)
(73, 1571)
(73, 2956)
(76, 1216)
(92, 2104)
(92, 2577)
(102, 2577)
(139, 2577)
(139, 2867)
(152, 2956)
(162, 2577)


In [9]:
# hyperparameter tuning
param_dict={}
space = {
        'max_depth': hp.choice('max_depth', np.arange(1, 20, dtype=int)),
        'min_child_weight': hp.choice('min_child_weight', np.arange(1, 20, dtype=int)),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'n_estimators': hp.choice('n_estimators', np.arange(10, 1000, 10, dtype=int)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        'gamma': hp.uniform('gamma', 0, 10),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}


def objective(space):
    params = {
            'min_child_weight': space['min_child_weight'],
            'eta': space['learning_rate'],
            'colsample_bytree': space['colsample_bytree'],
            'max_depth': space['max_depth'],
            'subsample': space['subsample'],
            'gamma': space['gamma'],
            'n_estimators': space['n_estimators'],
            'silent': 1,
            'verbose_eval': True

}
    print(params)
    count=0
    X_train=data_fold1
    X_test=valid_fold1
    y_train=X_train['num_orders']
    y_train=np.log1p(y_train)
    y_test=X_test['num_orders']
    y_test=np.log1p(y_test)
    X_train=X_train.drop(['num_orders','week'],axis=1)
    X_test=X_test.drop(['num_orders','week'],axis=1) 
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params,xgtrain,space['n_estimators'])
    count=count+np.sqrt(mean_squared_error(y_test,model.predict(xgb.DMatrix(X_test)))) 
    X_train=data_fold2
    X_test=valid_fold2
    y_train=X_train['num_orders']
    y_train=np.log1p(y_train)
    y_test=X_test['num_orders']
    y_test=np.log1p(y_test)
    X_train=X_train.drop(['num_orders','week'],axis=1)
    X_test=X_test.drop(['num_orders','week'],axis=1)
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params,xgtrain,space['n_estimators'])
    count=count+np.sqrt(mean_squared_error(y_test,model.predict(xgb.DMatrix(X_test)))) 
    X_train=data_fold3
    X_test=valid_fold3
    y_train=X_train['num_orders']
    y_train=np.log1p(y_train)
    y_test=X_test['num_orders']
    y_test=np.log1p(y_test)
    X_train=X_train.drop(['num_orders','week'],axis=1)
    X_test=X_test.drop(['num_orders','week'],axis=1)
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params,xgtrain,space['n_estimators'])
    count=count+np.sqrt(mean_squared_error(y_test,model.predict(xgb.DMatrix(X_test)))) 
    X_train=data_fold4
    X_test=valid_fold4
    y_train=X_train['num_orders']
    y_train=np.log1p(y_train)
    y_test=X_test['num_orders']
    y_test=np.log1p(y_test)
    X_train=X_train.drop(['num_orders','week'],axis=1)
    X_test=X_test.drop(['num_orders','week'],axis=1)
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params,xgtrain,space['n_estimators'])
    count=count+np.sqrt(mean_squared_error(y_test,model.predict(xgb.DMatrix(X_test)))) 
    X_train=data_fold5
    X_test=valid_fold5
    y_train=X_train['num_orders']
    y_train=np.log1p(y_train)
    y_test=X_test['num_orders']
    y_test=np.log1p(y_test)
    X_train=X_train.drop(['num_orders','week'],axis=1)
    X_test=X_test.drop(['num_orders','week'],axis=1)
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params,xgtrain,space['n_estimators'])
    count=count+np.sqrt(mean_squared_error(y_test,model.predict(xgb.DMatrix(X_test)))) 
    print(count/5)
    return count/5
trials = Trials()
best = hyperopt.fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials
)

{'min_child_weight': 11, 'eta': 0.054112783152834326, 'colsample_bytree': 0.9329331790770039, 'max_depth': 10, 'subsample': 0.5745854793052987, 'gamma': 5.391851614290842, 'n_estimators': 350, 'silent': 1, 'verbose_eval': True}
0.5466840016312732
{'min_child_weight': 5, 'eta': 0.028734744374393828, 'colsample_bytree': 0.7009722443384222, 'max_depth': 15, 'subsample': 0.9346297845667467, 'gamma': 0.5209839177014652, 'n_estimators': 820, 'silent': 1, 'verbose_eval': True}
0.554201102568686
{'min_child_weight': 18, 'eta': 0.02304043995048209, 'colsample_bytree': 0.601428014075881, 'max_depth': 9, 'subsample': 0.6861674838511556, 'gamma': 2.4492315069822324, 'n_estimators': 390, 'silent': 1, 'verbose_eval': True}
0.5420861227682197
{'min_child_weight': 18, 'eta': 0.053302215267295144, 'colsample_bytree': 0.6743660003905195, 'max_depth': 6, 'subsample': 0.7210425740673332, 'gamma': 6.189944665472912, 'n_estimators': 430, 'silent': 1, 'verbose_eval': True}
0.5464210845073408
{'min_child_weig

KeyboardInterrupt: 

In [ ]:
# best params
# {'min_child_weight': 13, 'eta': 0.03958887741804744, 'colsample_bytree': 0.7443787645516902, 'max_depth': 8, 'subsample': 0.9021398533405871, 'gamma': 1.2301766680324495, 'n_estimators': 410, 'silent': 1, 'verbose_eval': True}
# 0.5413848298152978

In [11]:
X_train=X_MAIN.drop(['num_orders','week'],axis=1)
Y=np.log1p(y_MAIN)
X_test=test.drop(['num_orders','week'],axis=1)

In [14]:
space={'min_child_weight': 13, 'eta': 0.03958887741804744, 'colsample_bytree': 0.7443787645516902, 'max_depth': 8, 'subsample': 0.9021398533405871, 'gamma': 1.2301766680324495, 'n_estimators': 410, 'silent': 1, 'verbose_eval': True}
xgtrain = xgb.DMatrix(X, label=Y)
model = xgb.train(space,xgtrain,space['n_estimators'])

In [15]:
preds=model.predict(xgb.DMatrix(X_test))

In [35]:
df=pickle.load(open('training_data','rb'))
df=df[df.week>=146]
ids=df['id']

In [36]:
sub=pd.DataFrame()
sub['id']=ids
sub['num_orders']=list(np.expm1((preds)))

In [38]:
sub.to_csv('submission.csv',index=False)